### Import Packages

In [15]:
%matplotlib inline

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

import math 
import numpy as np
import csv
import pandas as pd

import sys
import os

from tqdm import tqdm

print(torch.__version__)
torch.set_default_tensor_type(torch.FloatTensor)

1.8.0+cu111


### Define Some utility Functions

In [34]:
def same_seed(seed):
    '''
        Fixes random number generator seeds for reproducibility
    '''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(dataset, valid_ratio, seed, print_shape=False):
    valid_set_size = int(valid_ratio * dataset.shape[0])
    train_set_size = dataset.shape[0] - valid_set_size
    
    train_data, valid_data= random_split(dataset, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    
    train_data = np.array(train_data)
    valid_data = np.array(valid_data)
    if print_shape:
        print(f"Origin Train Set size: {train_data.shape}, Origin Valid Set Size: {valid_data.shape}")
    
    return train_data, valid_data

def predict(valid_loader, model, device):
    model.eval()
    preds = []
    ys = []
    for x, y in tqdm(valid_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
            ys.append(y.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    ys    = torch.cat(ys, dim=0).numpy()
    return preds, ys


### Dataset

In [17]:
class CTCRDataset(Dataset):
    '''
    x: features,
    y: Targets, if none, do prediction
    '''
    
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        
        self.x = torch.FloatTensor(x)
        
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]
        
    def __len__(self):
        return len(self.x)    
        

### Neural Network Model

In [19]:
class FK_MLP(nn.Module):
    
    def __init__(self, target_type="pose"):
        super(FK_MLP, self).__init__()
        
        self.target_type = target_type
        
        if target_type == "3_points_pos":
            self.output_dims = 9
        elif target_type == "end_pos":
            self.output_dims = 3
        else:
            self.output_dims = 7
            
        self.layers = nn.Sequential(
            nn.Linear(12, 128),
            nn.ReLU(),
            
            nn.Linear(128, 256),
            nn.ReLU(),
            
            nn.Linear(256, 512),
            nn.ReLU(),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            
            nn.Linear(256, 64),
            nn.ReLU(),
            
            nn.Linear(64, self.output_dims)
        )
        
    def forward(self, x):
        y = self.layers(x)
        return y

### Training Loop

In [20]:
def loss_pos(preds, labels, reduction="mean"):
    if reduction == "mean": 
        loss = torch.mean((preds - labels) ** 2)
    else:
        loss = torch.sum((preds - labels) ** 2)
        
    return loss
        
def loss_orientation(preds, labels):
    loss = torch.acos(torch.sum(preds * labels))
    return loss
    
def loss_func(preds, labels):
    pass
    

def trainer(train_loader, valid_loader, model, config, device):
    
#     criterion = loss_pos()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    writer = SummaryWriter()
    
    if not os.path.isdir("./checkpoints"):
        os.mkdir("./checkpoints")
        
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    
    for epoch in range(n_epochs):
        model.train()
        loss_record = []
        
        train_pbar = tqdm(train_loader, position=0, leave=True)
        
        for x, y in train_pbar:
            optimizer.zero_grad()
            
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = loss_pos(pred, y, reduction="sum")
            loss.backward()
            
            optimizer.step()
            
            step += 1
            loss_record.append(loss.detach().item())
            
#             train_pbar.set_description(f"[{epoch+1}/{n_epochs}]")
            train_pbar.set_description(f"{epoch+1}")
            train_pbar.set_postfix({'loss': loss.detach().item()})
        
#         print(loss_record)
            
        mean_train_loss = sum(loss_record) / len(loss_record)
        writer.add_scalar('Loss/Train', mean_train_loss, step)
        
        model.eval()
        loss_record = []
        
        
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = loss_pos(pred, y, reduction="sum")
            loss_record.append(loss.item())
        
        
        mean_valid_loss = sum(loss_record) / len(loss_record)
        
        if mean_valid_loss <= best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])
            print("Saving Model with loss {:.3f}...".format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1
        
        if early_stop_count >= config['early_stop']:
            print("\nModel is not improving, so we halt the training sessions")
            
            return 
        
                

### Config

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"

config = {
    'seed': 10,
    'valid_ratio': 0.2,
    'n_epochs': 100,
    'batch_size': 8,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'early_stop': 100,
    'save_path': "./checkpoints/model.ckpt",
    "features_type": "all",
    "target_type": "end_pos",
}

### Data Process And Dataloader

#### Select Features

In [30]:
L1 = 210
L2 = 165
L3 = 110
L = [L1, L2, L3]

Mb = np.array([
    [-L1, 0,     0], 
    [-L1, L1-L2, 0],
    [-L1, L1-L2, L2-L3]
])

T = np.zeros((4, 4))
T[0:3, 0:3] = 1/2 * Mb
T[0:3, 3:]  = 1/2 * np.dot(Mb, np.ones((3, 1)))
T[3, 3] = 1


invT = np.linalg.inv(T)

def normalize_beta(origin_train_data, print_shape=False):
    origin_train_beta = origin_train_data[:, [1, 3, 5, 6]]
    origin_train_beta[:, 3:] = np.ones((origin_train_beta.shape[0], 1))
    norm_beta = np.transpose(np.dot(invT, np.transpose(origin_train_beta)))
    
    
    origin_train_delta_beta = origin_train_data[:, [7, 9, 11, 12]]
    origin_train_delta_beta[:, 3:] = np.ones((origin_train_delta_beta.shape[0], 1))
    norm_delta_beta = np.transpose(np.dot(invT, np.transpose(origin_train_delta_beta)))
    
    if print_shape:
        print(f"Normalized beta Shape : {norm_beta.shape}, Normalized delta beta Shape: {norm_delta_beta.shape}")
        
    return norm_beta[:, 0:3], norm_delta_beta[:, 0:3]    


def select_features_target(origin_train_data, origin_valid_data, test_data=None, features_type="all", target_type="pose", print_shape=False):
    train_data = np.zeros((origin_train_data.shape[0], 12))
    valid_data = np.zeros((origin_valid_data.shape[0], 12))
   
    for i in range(0, 6, 2):
        train_data[:, 4*(i//2)  ] = np.cos(origin_train_data[:, i])
        train_data[:, 4*(i//2)+1] = np.sin(origin_train_data[:, i])
        train_data[:, 4*(i//2)+2] = origin_train_data[:, i+1]
        train_data[:, 4*(i//2)+3:] = np.ones((origin_train_data.shape[0], 1)) * L[i//2]

        valid_data[:, 4*(i//2)  ] = np.cos(origin_valid_data[:, i])
        valid_data[:, 4*(i//2)+1] = np.sin(origin_valid_data[:, i])
        valid_data[:, 4*(i//2)+2] = origin_valid_data[:, i+1]
        valid_data[:, 4*(i//2)+3:] = np.ones((origin_valid_data.shape[0], 1)) * L[i//2]

#     train_norm_beta, train_nrom_delta_beta = normalize_beta(origin_train_data, print_shape=True)
#     valid_norm_beta, valid_norm_delta_beta = normalize_beta(origin_valid_data, print_shape=True)
    
#     assert train_data.shape[0] == train_norm_beta.shape[0], "Shape Error"
#     assert valid_data.shape[0] == valid_norm_beta.shape[0], "Shape Error"
    
#     for i in range(3):
#         train_data[:, 3*i+2] = train_norm_beta[:, i]
# #         train_data[:, 3*i+11] = train_nrom_delta_beta[:, i]
        
#         valid_data[:, 3*i+2] = valid_norm_beta[:, i]
# #         valid_data[:, 3*i+11] = valid_norm_delta_beta[:, i]

    
    feature_idx = []
    if features_type == "all":
        feature_idx = list(range(train_data.shape[1]))
    else:
        pass
    
    if target_type == "3_points_pos":
        train_target = np.concatenate((origin_train_data[:, 19:22], origin_train_data[:, 26:29], origin_train_data[:, -7:-4]), axis=1)
        train_target = np.concatenate((origin_valid_data[:, 19:22], origin_valid_data[:, 26:29], origin_valid_data[:, -7:-4]), axis=1)
    elif target_type == "end_pos":
        train_target = origin_train_data[:, -7:-4]
        valid_target = origin_valid_data[:, -7:-4]
    else:
        train_target = origin_train_data[:, -7:]
        valid_target = origin_valid_data[:, -7:]
    
    if print_shape:
        print(f"Train data Shape: {train_data.shape}, Valid data shape: {valid_data.shape}")
        print(f"Target Type = {target_type}, \nTrain Target Shape: {train_target.shape}, Valid Target Shape: {valid_target.shape}")
    
    if test_data == None:
        return train_data, train_target, valid_data, valid_target
    else:
        return train_data, train_target, valid_data, valid_target, test_data

#### Dataloader

In [31]:
same_seed(config['seed'])
path = "./dataset/CRL-Dataset-CTCR-Pose.csv"

origin_data = pd.read_csv(path, header=None).values
origin_train_data, origin_valid_data = train_valid_split(origin_data, valid_ratio=config['valid_ratio'], seed=config['seed'], print_shape=True)

train_data, train_target, valid_data, valid_target = select_features_target(origin_train_data, origin_valid_data, test_data=None, features_type=config['features_type'], target_type=config['target_type'], print_shape=True)

train_dataset, valid_dataset = CTCRDataset(train_data, train_target), CTCRDataset(valid_data, valid_target)
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)


Origin Train Set size: (80000, 40), Origin Valid Set Size: (20000, 40)
Train data Shape: (80000, 12), Valid data shape: (20000, 12)
Target Type = end_pos, 
Train Target Shape: (80000, 3), Valid Target Shape: (20000, 3)


In [32]:
model = FK_MLP(target_type="end_pos").cuda()
# model.load_state_dict(torch.load("./checkpoints/model.ckpt"))
trainer(train_loader, valid_loader, model, config, device)


1: 100%|██████████████████████████████████████████████████████████| 10000/10000 [01:04<00:00, 155.42it/s, loss=1.04e+3]


Saving Model with loss 5304.410...


2: 100%|██████████████████████████████████████████████████████████████| 10000/10000 [01:03<00:00, 158.30it/s, loss=134]


Saving Model with loss 242.878...


4: 100%|██████████████████████████████████████████████████████████████| 10000/10000 [01:04<00:00, 156.16it/s, loss=223]


...

Saving Model with loss 19.431...


70: 100%|████████████████████████████████████████████████████████████| 10000/10000 [00:54<00:00, 184.70it/s, loss=3.57]


Saving Model with loss 18.846...


85:  78%|███████████████████████████████████████████████▍             | 7785/10000 [00:51<00:14, 151.42it/s, loss=4.52]


KeyboardInterrupt: 

In [35]:
def save_pred(preds, labels=None, file="./checkpoints/pred.csv"):
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        
        n = preds.shape[0]
        if labels is not None:
            for i in range(n):
                ls = []
                error_x = np.sqrt(np.sum(np.square(labels[i] - preds[i])))
                ls.append(error_x)
                writer.writerow(np.concatenate((preds[i], labels[i], labels[i] - preds[i], (labels[i] - preds[i]) / labels[i] * 100, ls)))
        else :
            for i1,p1 in enumerate(preds):
                writer.writerow(p2)

model.load_state_dict(torch.load(config['save_path']))
model.eval()
preds, labels = predict(valid_loader, model, device)

save_pred(preds, labels, './checkpoints/pred.csv')

train_pred, train_labels = predict(train_loader, model, device)

save_pred(train_pred, train_labels, './checkpoints/train_preds.csv')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:08<00:00, 1129.30it/s]


In [ ]:
print(type(preds), type(valid_target))
print(preds.shape, valid_target.shape)